In [13]:
import pandas as pd

# CSV is in the same folder as the notebook
df = pd.read_csv("anomaly_data.csv")

print(df.shape)
df.head()


(5000, 7)


,TransactionID,CustomerAge,TransactionAmount,TransactionTimeHour,NumPrevTransactions,LocationRiskScore,IsFraud
0,1,56,316.02,15,17,0.95,0
1,2,69,50.92,19,45,0.37,0
2,3,46,183.13,5,24,0.83,0
3,4,32,20.89,10,27,0.19,0
4,5,60,111.61,23,7,0.81,0


In [14]:
print(df.columns.tolist())
df.describe(include="all")


['TransactionID', 'CustomerAge', 'TransactionAmount', 'TransactionTimeHour', 'NumPrevTransactions', 'LocationRiskScore', 'IsFraud']


,TransactionID,CustomerAge,TransactionAmount,TransactionTimeHour,NumPrevTransactions,LocationRiskScore,IsFraud
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,43.584600,97.259972,11.742800,24.950200,0.507560,0.035200
std,1443.520003,14.919094,97.444622,6.827442,14.264544,0.287437,0.184303
min,1.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1250.750000,31.000000,27.675000,6.000000,13.000000,0.260000,0.000000
50%,2500.500000,43.000000,66.610000,12.000000,25.000000,0.510000,0.000000
75%,3750.250000,56.000000,134.502500,18.000000,37.000000,0.750000,0.000000
max,5000.000000,69.000000,772.350000,23.000000,49.000000,1.000000,1.000000


In [16]:
from pycaret.anomaly import *

anom_exp = setup(
    data=df,
    ignore_features=["TransactionID"],   # ID column
    session_id=123,
    normalize=True,                      # scale features
    verbose=False                        # hides extra logs; replaces old 'silent=True'
)


In [17]:
iforest = create_model('iforest')
labeled_df = assign_model(iforest)
labeled_df.head()


,CustomerAge,TransactionAmount,TransactionTimeHour,NumPrevTransactions,LocationRiskScore,IsFraud,Anomaly,Anomaly_Score
0,56,316.019989,15,17,0.95,0,0,-0.057221
1,69,50.919998,19,45,0.37,0,0,-0.071065
2,46,183.130005,5,24,0.83,0,0,-0.117112
3,32,20.889999,10,27,0.19,0,0,-0.143919
4,60,111.610001,23,7,0.81,0,0,-0.055577


In [18]:
# Train the Isolation Forest model
iforest = create_model('iforest')
iforest


IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)

In [19]:
# Label anomalies in the dataset
labeled_df = assign_model(iforest)
labeled_df.head(20)


,CustomerAge,TransactionAmount,TransactionTimeHour,NumPrevTransactions,LocationRiskScore,IsFraud,Anomaly,Anomaly_Score
0,56,316.019989,15,17,0.95,0,0,-0.057221
1,69,50.919998,19,45,0.37,0,0,-0.071065
2,46,183.130005,5,24,0.83,0,0,-0.117112
3,32,20.889999,10,27,0.19,0,0,-0.143919
4,60,111.610001,23,7,0.81,0,0,-0.055577
5,25,377.260010,2,9,0.66,0,0,-0.020055
6,38,10.750000,19,43,0.19,0,0,-0.110697
7,56,0.840000,9,35,0.30,0,0,-0.129521
8,36,56.840000,0,12,0.13,0,0,-0.085027
9,40,9.720000,19,21,0.38,0,0,-0.144841


In [20]:
# Compare model's anomalies with real fraud labels
actual_frauds = (labeled_df["IsFraud"] == 1).sum()
model_anomalies = (labeled_df["Anomaly"] == 1).sum()
overlap = labeled_df[(labeled_df["IsFraud"] == 1) & (labeled_df["Anomaly"] == 1)].shape[0]

print(f"✅ Actual frauds in data: {actual_frauds}")
print(f"✅ Anomalies flagged by model: {model_anomalies}")
print(f"✅ Frauds correctly identified as anomalies: {overlap}")


✅ Actual frauds in data: 176
✅ Anomalies flagged by model: 250
✅ Frauds correctly identified as anomalies: 176


In [21]:
# Save the trained model for later use
save_model(iforest, "anomaly_iforest_model")


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['CustomerAge', 'TransactionAmount',
                                              'TransactionTimeHour',
                                              'NumPrevTransactions',
                                              'LocationRiskScore', 'IsFraud'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('trained_model',
                  IForest(behaviour='new', bootstrap=False, contamination=0.05,
     max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
     random_state=123, verbose=0))]),
 'anomaly_iforest_model.pkl')

In [22]:
plot_model(iforest, plot='tsne')
